# About 
In this notebook, we build a CNN network and trains it with CIFAR10 dataset

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data

import torchvision
from torchvision import transforms
from torchvision import datasets

from my_nn_models import CNNCifar
from data_preparation import data_setup


In [7]:
class HyperParam():
    def __init__(self, path='..\data\cifar',learning_rate=0.1, batch_size=64, epoch=100, momentum=True, nesterov=False):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.datapath = path
        self.lr=learning_rate
        self.bs=batch_size
        self.epoch=epoch
        self.momentum=momentum
        self.nesterov=nesterov

settings = HyperParam(batch_size=4)
model = CNNCifar().to(settings.device)
loss_fn = nn.CrossEntropyLoss().to(settings.device)
if settings.nesterov:
    optimizer = torch.optim.SGD(model.parameters(), lr=settings.lr, momentum=settings.momentum, nesterov=settings.nesterov)
else:
    optimizer = torch.optim.SGD(model.parameters(), lr=settings.lr)
print(settings.lr,settings.bs, settings.epoch, settings.momentum, settings.device, settings.datapath)
print(model)
print(loss_fn)
settings.epoch

0.1 4 100 True cuda ..\data\cifar
CNNCifar(
  (conv_layer): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layer): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)
CrossEntropyLoss()


100

In [3]:
loader_train, loader_test = data_setup(path=settings.datapath,batch_size=settings.bs)
# check out the imagesize
train_iter = iter(loader_train)
images, labels = train_iter.next()
print(labels)
# len(loader_train.dataset)

tensor([7, 3, 9, 9])


In [4]:
# define loss and optimizer
loss_fn = nn.CrossEntropyLoss().to(settings.device)
optimizer = torch.optim.SGD(model.parameters(), lr=settings.lr, momentum=0.9, nesterov=True)

# # start training
# for epoch in range(1, epochs+1):
#     train_loss = 0.0
#     test_acc = 0.0
#     model.train()
#     for batch, (images, labels) in enumerate(loader_train):
#         images, labels = images.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = loss_fn(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         train_loss+=train_loss.item() * images.size(0)
#     epoch_loss = train_loss/len(loader_train.dataset)



In [5]:
import os
os.getcwd()

'C:\\Users\\wangyuan\\my-torch-handson\\Pytorch-and-DL'